## Example of training using the Alpha Neuron

In [ ]:
# !git clone -b alpha_neuron --single-branch https://github.com/jeshraghian/snntorch.git
!pip install snntorch

fatal: destination path 'snntorch' already exists and is not an empty directory.


In [ ]:
# %cd snntorch
import snntorch as snn

/content/snntorch/snntorch


## Import Packages

In [ ]:
import snntorch as snn
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import itertools
import matplotlib.pyplot as plt

## Define Network and Parameters

In [ ]:
# test alpha neuron: can it learn?

num_inputs = 28*28
num_hidden = 1000
num_outputs = 10

# Training Parameters
batch_size=128
data_path='/data/mnist'

# Temporal Dynamics
num_steps = 25
alpha = 0.9
beta = 0.8

dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Define Network
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        # initialize layers
        self.fc1 = nn.Linear(num_inputs, num_hidden)
        self.lif1 = snn.Alpha(alpha=alpha, beta=beta)
        self.fc2 = nn.Linear(num_hidden, num_outputs)
        self.lif2 = snn.Alpha(alpha=alpha, beta=beta)


    def forward(self, x):
        spk1, syn_exc1, syn_inh1, mem1 = self.lif1.init_alpha(batch_size, num_hidden)
        spk2, syn_exc2, syn_inh2, mem2 = self.lif2.init_alpha(batch_size, num_outputs)

        # Record the final layer
        spk2_rec = []
        mem2_rec = []

        for step in range(num_steps):

            cur1 = self.fc1(x)
            spk1, syn_exc1, syn_inh1, mem1 = self.lif1(cur1, syn_exc1, syn_inh1, mem1)
            cur2 = self.fc2(spk1)
            spk2, syn_exc2, syn_inh2, mem2 = self.lif2(cur2, syn_exc2, syn_inh2, mem2)

            spk2_rec.append(spk2)
            mem2_rec.append(mem2)

        return torch.stack(spk2_rec, dim=0), torch.stack(mem2_rec, dim=0)
  
net = Net().to(device)

## dataloaders

In [ ]:
# Define a transform
transform = transforms.Compose([
            transforms.Resize((28, 28)),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))])

mnist_train = datasets.MNIST(data_path, train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(data_path, train=False, download=True, transform=transform)

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
# Create DataLoaders
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=True, drop_last=True)

## Print Accuracy Function

In [ ]:
def print_batch_accuracy(data, targets, train=False):
    with torch.no_grad():
      output, _ = net(data.view(batch_size, -1))
      _, idx = output.sum(dim=0).max(1)
      acc = np.mean((targets == idx).detach().cpu().numpy())

      if train:
          print(f"Train Set Accuracy: {acc}")
      else:
          print(f"Test Set Accuracy: {acc}")

def train_printer():
    print(f"Epoch {epoch}, Minibatch {minibatch_counter}")
    print(f"Train Set Loss: {loss_hist[counter]}")
    print(f"Test Set Loss: {test_loss_hist[counter]}")
    print_batch_accuracy(data_it, targets_it, train=True)
    print_batch_accuracy(testdata_it, testtargets_it, train=False)
    print("\n")

## Define Loss & Optimizer

In [ ]:
optimizer = torch.optim.Adam(net.parameters(), lr=2e-4, betas=(0.9, 0.999))
log_softmax_fn = nn.LogSoftmax(dim=-1)
loss_fn = nn.NLLLoss()

## Training Loop

In [ ]:
loss_hist = []
test_loss_hist = []
counter = 0

# Outer training loop
for epoch in range(10):
    minibatch_counter = 0
    train_batch = iter(train_loader)

    # Minibatch training loop
    for data_it, targets_it in train_batch:
        data_it = data_it.to(device)
        targets_it = targets_it.to(device)

        spk_rec, mem_rec = net(data_it.view(batch_size, -1))
        log_p_y = log_softmax_fn(mem_rec)
        loss_val = torch.zeros((1), dtype=dtype, device=device)

        # Sum loss over time steps: BPTT
        for step in range(num_steps):
          loss_val += loss_fn(log_p_y[step], targets_it)

        # Gradient calculation
        optimizer.zero_grad()
        loss_val.backward()

        # Weight Update
        optimizer.step()

        # Store loss history for future plotting
        loss_hist.append(loss_val.item())

        # Test set
        test_data = itertools.cycle(test_loader)
        testdata_it, testtargets_it = next(test_data)
        testdata_it = testdata_it.to(device)
        testtargets_it = testtargets_it.to(device)

        # Test set forward pass
        with torch.no_grad():
          test_spk, test_mem = net(testdata_it.view(batch_size, -1))

          # Test set loss
          log_p_ytest = log_softmax_fn(test_mem)
          log_p_ytest = log_p_ytest.sum(dim=0)
          loss_val_test = loss_fn(log_p_ytest, testtargets_it)
          test_loss_hist.append(loss_val_test.item())

          # Print test/train loss/accuracy
          if counter % 50 == 0:
              train_printer()
          minibatch_counter += 1
          counter += 1

loss_hist_true_grad = loss_hist
test_loss_hist_true_grad = test_loss_hist

Epoch 0, Minibatch 0
Train Set Loss: 57.23897171020508
Test Set Loss: 56.607032775878906
Train Set Accuracy: 0.1171875
Test Set Accuracy: 0.0625


Epoch 0, Minibatch 50
Train Set Loss: 39.31348419189453
Test Set Loss: 37.7827033996582
Train Set Accuracy: 0.3203125
Test Set Accuracy: 0.3046875


Epoch 0, Minibatch 100
Train Set Loss: 27.320207595825195
Test Set Loss: 26.991588592529297
Train Set Accuracy: 0.40625
Test Set Accuracy: 0.390625


Epoch 0, Minibatch 150
Train Set Loss: 25.243783950805664
Test Set Loss: 24.81044578552246
Train Set Accuracy: 0.390625
Test Set Accuracy: 0.296875


Epoch 0, Minibatch 200
Train Set Loss: 26.74785614013672
Test Set Loss: 24.078622817993164
Train Set Accuracy: 0.3359375
Test Set Accuracy: 0.390625


Epoch 0, Minibatch 250
Train Set Loss: 21.86463737487793
Test Set Loss: 20.568628311157227
Train Set Accuracy: 0.3515625
Test Set Accuracy: 0.3515625


Epoch 0, Minibatch 300
Train Set Loss: 20.57036590576172
Test Set Loss: 20.193082809448242
Train Set 

KeyboardInterrupt: ignored